In [1]:
pip install PyMuPDF chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.0 MB/s eta 0:00:00
   

In [2]:
import fitz

In [5]:
#Extract the text form pdf
def extract_text_from_pdf(pdf_path, num_pages=5):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(min(num_pages, doc.page_count)):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

In [4]:
pdf_path = "/content/budget_speech.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

In [7]:
print(pdf_text[:500])

GOVERNMENT OF INDIA
BUDGET 2023-2024
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2023
CONTENTS 
PART-A 
Page No.
 
Introduction 
1
 
Achievements since 2014: Leaving no one behind 
2
 
Vision for Amrit Kaal – an empowered and inclusive economy 
3
 
Priorities of this Budget 
5
i. Inclusive Development  
ii. Reaching the Last Mile 
iii. Infrastructure and Investment 
iv. Unleashing the Potential 
v. Green Growth 
vi. Youth Power  
vii. Financial Sector 
 
 
 
 
 
 
 
 
 
Fi


In [8]:
#Creating a vector database and i am using chromadb as pinecone give me some authentication errors and search for that and i dont get the answer
import chromadb

In [9]:
# Initialize Chroma client
client = chromadb.Client()

In [10]:
#creating collection for storing the data
collection = client.create_collection("business_bot_pdf")

In [11]:
#creating chunks of data
chunks = pdf_text.split("\n")

In [12]:
#adding chunks to the collection of chromadb
for idx, chunk in enumerate(chunks):
    if chunk.strip():
        collection.add(
            documents=[chunk],
            metadatas=[{"source": f"chunk_{idx}"}],
            ids=[f"chunk_{idx}"]
        )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 92.6MiB/s]


In [15]:
#Creating embeddings
import numpy as np
from sentence_transformers import SentenceTransformer


In [16]:
#loading a pretrained transformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
#function to create embedding text
def embed_text(text):
    embedding_tensor = embedding_model.encode(text, convert_to_tensor=True)
    embedding_numpy = embedding_tensor.detach().cpu().numpy()
    return embedding_numpy

In [18]:
embeddings = []#list to store the values

In [19]:
for chunk in chunks:
    if chunk.strip():
        embeddings.append(embed_text(chunk))

In [20]:
#adding the embeddings into the chromadb collections
for idx, embedding in enumerate(embeddings):
    collection.add(
        documents=[chunks[idx]],
        metadatas=[{"source": f"chunk_{idx}"}],
        ids=[f"chunk_{idx}"],
        embeddings=[embedding]
    )

In [21]:
#function to retireve relevant documents based on the query
def retrieve_relevant_docs(query: str):
    query_embedding = embed_text(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=3)
    documents = results.get("documents", [])
    return [doc for doc in documents]

In [22]:
#Example usuage
query = "What are the key priorities outlined in the 2023-2024 Budget speech by Nirmala Sitharaman, and how do they align with the vision for an empowered and inclusive economy?"
retrieved_docs = retrieve_relevant_docs(query)
print("Retrieved Documents:", retrieved_docs)

Retrieved Documents: [['Priorities of this Budget ', 'Annexure to Part B of the Budget Speech 2023-24 ', 'BUDGET 2023-2024']]


In [23]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

In [24]:
#function to flatten
def flatten_and_join(documents):
    flat_documents = []
    for item in documents:
        if isinstance(item, list):
            flat_documents.extend(flatten_and_join(item))
        else:
            flat_documents.append(str(item))
    return flat_documents

In [25]:
#function to call the hugging face model and i am using this because my openai free trails are over and it said to upgrade to get the credits so for that i am using this free model from hugging face.
def generate_answer_with_bert(documents, query):
    documents = flatten_and_join(documents)
    context = " ".join(documents)
    if not context or not query:
        return "Error: Context or query is empty."
    #Loading the BERT tokenizer and model for question answering
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    #tokinizing the input
    inputs = tokenizer.encode_plus(query, context, return_tensors="pt", add_special_tokens=True)
    with torch.no_grad():
        output = model(**inputs)
    start_scores = output.start_logits
    end_scores = output.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    #decoding the answer using the tokenizer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
    return answer

In [30]:
#example query
query = "What are your ai services?"


In [31]:
#calling the genrated fucntion
generated_answer = generate_answer_with_bert(retrieved_docs, query)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
print("Generated Answer:", generated_answer)

Generated Answer: ai services ? [SEP]
